In [1]:
import numpy as np
from skimage import io, transform
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision import models

from tqdm import tqdm

import pandas as pd
import os

%matplotlib inline

/home/sphere_7/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('marks.csv', header=None)
data.head()

,0,1
0,Gesture_4/WIN_20171129_12_29_10_Pro (2).jpg,4
1,Gesture_4/2017-11-26-145636_9.jpg,4
2,Gesture_4/WIN_20171203_17_26_42_Pro.jpg,4
3,Gesture_4/2017-11-26-150319_4.jpg,4
4,Gesture_4/2017-11-26-150319_9.jpg,4


In [3]:
class GesturesDataset(Dataset):
    def __init__(self, pd_file, root_dir, transform=None):
        self.marks = pd_file
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.marks)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.marks.iloc[idx, 0])
        image = io.imread(img_name)
        sample = {'image': image, 'mark': self.marks.iloc[idx, 1] - 1}

#         if self.transform:
#             sample = self.transform(sample)

        return sample

In [4]:
train_data, test_data = train_test_split(data, test_size=0.2)

In [5]:
train_dataset = GesturesDataset(pd_file=train_data, root_dir='./')
train_dataloader = DataLoader(dataset=train_dataset, batch_size=10, num_workers=2, shuffle=True)

test_dataset = GesturesDataset(pd_file=test_data, root_dir='./')
test_dataloader = DataLoader(dataset=test_dataset, batch_size=10, num_workers=2, shuffle=True)

In [6]:
alex = models.AlexNet().features.cuda()

In [58]:
X.size()

torch.Size([10, 720, 1280, 3])

In [61]:
X = X.permute(0, 3, 1, 2)

In [63]:
X = Variable(X.float()).cuda()

# X = Variable(X, requires_grad=True).cuda()

In [65]:
alex(X).data.view(X.size(0), -1).size()         

torch.Size([10, 209664])

In [10]:
c = torch.load('../alex-checkpoint-82.pth.tar')

In [15]:
model = nn.Sequential(nn.Linear(209664, 5), nn.LogSoftmax()).cuda()
loss_fn = nn.NLLLoss()

In [14]:
model.load_state_dict(c['state_dict'])

In [16]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [17]:
def emulate(N, model, optimizer, loss_fn):
    n_epoch = N

    losses = np.zeros(n_epoch)
    test_losses = np.zeros(n_epoch)

    for i in tqdm(range(n_epoch)):
        for batch in train_dataloader:
            X = batch['image']
            y = batch['mark']
            
            X = X.permute(0, 3, 1, 2)
            X = Variable(X.float()).cuda()
            X = alex(X).data.view(X.size(0), -1)
            
            X = Variable(X, requires_grad=True).cuda()
            y = Variable(y.float(), requires_grad=False).cuda()
    
            y_pred = model(X)
            loss = loss_fn(y_pred, y.long())
            losses[i] = max(loss.data[0], losses[i])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
#         for batch in test_dataloader:
#             X = batch['image']
#             y = batch['mark']
            
#             X = X.permute(0, 3, 1, 2)
#             X = Variable(X.float()).cuda()
#             X = alex(X).data.view(X.size(0), -1)
            
#             X = Variable(X, requires_grad=True).cuda()
#             y = Variable(y.float(), requires_grad=False).cuda()
    
#             y_pred = model(X)
#             loss = loss_fn(y_pred, y.long())
#             test_losses[i] = max(loss.data[0], test_losses[i])            
        if i != 0:
            if losses[i] < 20 and losses[i] > 0:
                print(losses[-1])
                break
    return losses, test_losses

In [18]:
losses, test_losses = emulate(100, model, optimizer, loss_fn)

 15%|█▌        | 15/100 [03:16<18:33, 13.10s/it]

0.0


In [19]:
losses

array([ 301.20953369,   81.49829865,   84.2124939 ,  126.70231628,
         34.97266006,   45.14321518,   29.21243286,   38.94561386,
         71.27126312,   30.93949509,   47.05743408,   49.33096313,
         38.77667236,   48.94181442,   39.97071075,   18.45581055,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [22]:
torch.save({
    'epoch': 16,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '../alex-checkpoint-80.pth.tar')

In [128]:
test_losses

array([ 619.74987793,  784.2668457 ])

In [16]:
list(np.array([1, 2, 3]))

[1, 2, 3]

In [27]:
test_loss = 0
pred = []
correct = []

for batch in test_dataloader:
    X = batch['image']
    y = batch['mark']
    
    correct += list(y.numpy())
    
    X = X.permute(0, 3, 1, 2)
    X = Variable(X.float()).cuda()
    X = alex(X).data.view(X.size(0), -1)
            
    X = Variable(X, requires_grad=True).cuda()
    y = Variable(y.float(), requires_grad=False).cuda()
    
    y_pred = model(X)
    loss = loss_fn(y_pred, y.long())
    print(accuracy_score(y.cpu().data.numpy(), y_pred.cpu().data.numpy().argmax(axis=1)))
    test_loss = max(loss.data[0], test_loss)

    
    pred += list(y_pred.cpu().data.numpy().argmax(axis=1))

0.7
1.0
0.8
0.9
0.8
0.9
0.8
0.9
0.9
0.8
0.8
0.9
0.5
0.7
0.5
0.9
0.8
0.8
1.0


In [28]:
accuracy_score(correct, pred)

0.80110497237569056

In [29]:
test_loss

20.81411361694336

In [48]:
tst

,0
0,2.jpg
1,3.jpg
2,5.jpg
3,1.jpg
4,4.jpg


In [47]:
tst = pd.DataFrame(data = os.listdir('../../test/test'))

In [49]:
tst['1'] = [3 for i in range(len(tst))]

In [50]:
tst_dataset = GesturesDataset(pd_file=tst, root_dir='../../test/test/')
tst_dataloader = DataLoader(tst_dataset, batch_size=1, num_workers=2, shuffle=False)

In [51]:
for batch in tst_dataloader:
    X = batch['image']
            
    X = X.permute(0, 3, 1, 2)
    X = Variable(X.float()).cuda()
    X = alex(X).data.view(X.size(0), -1)
            
    X = Variable(X, requires_grad=True).cuda()
    
    y_pred = model(X)
    
    print(y_pred)
    #plt.imshow(batch['image'][0].numpy())

Variable containing:
    0.0000 -1578.2942 -1112.2887  -263.2498  -791.4814
[torch.cuda.FloatTensor of size 1x5 (GPU 0)]

Variable containing:
    0.0000 -1621.5896 -1385.0001  -970.6466 -1708.4966
[torch.cuda.FloatTensor of size 1x5 (GPU 0)]

Variable containing:
-1642.6094     0.0000 -1743.7119  -312.9773  -958.2891
[torch.cuda.FloatTensor of size 1x5 (GPU 0)]

Variable containing:
-1698.8087 -1627.3885 -2301.3354     0.0000 -3129.5037
[torch.cuda.FloatTensor of size 1x5 (GPU 0)]

Variable containing:
-614.0206 -244.5977 -961.9930 -298.9932    0.0000
[torch.cuda.FloatTensor of size 1x5 (GPU 0)]



In [17]:
pred = np.zeros(len(test_dataset))
real = np.zeros(len(test_dataset))
for i in range(len(test_dataset)):
    X = torch.FloatTensor(test_dataset[i]['image'].astype(float).reshape(1, 720, 1280, 3))
    X = X.permute(0, 3, 1, 2)
    X = Variable(X.float()).cuda()
    X = alex(X).data.view(X.size(0), -1)
            
    X = Variable(X, requires_grad=True).cuda()
    
    y_pred = model(X)
    
    real[i] = test_dataset[i]['mark']
    pred[i] = y_pred.cpu().data.numpy().argmax()

In [18]:
accuracy_score(real, pred)

0.18439716312056736

In [101]:
test_dataset[0]['mark']

2

In [23]:
c = torch.load('../alex-checkpoint-80.pth.tar')

In [24]:
model = nn.Sequential(nn.Linear(209664, 5), nn.LogSoftmax()).cuda()
loss_fn = nn.NLLLoss()
model.load_state_dict(c['state_dict'])